In [17]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import time
from pathlib import Path
import atexit
from scipy.signal import butter, filtfilt
from scipy.stats import linregress
from scipy.signal import find_peaks
from scipy.ndimage import label 
import base64
import os
from pathlib import Path
from tqdm.notebook import tqdm


# ==== CONFIGURATION ====
INPUT_DIR = Path("/Users/Christian/Downloads/Javelin_Bad Krozingen und Neustadt_data")
OUTPUT_FILE = INPUT_DIR / "kinematic_summary.csv"
csv_files = list(INPUT_DIR.glob("*_merged.csv"))
all_results = []



# PARAMETERS:
# For javelin leaves the hand: 
MIN_DELTA = 100
MIN_FRACTION = 0.75
MIN_SUSTAIN_SEC = 0.2  # duration in seconds
min_sustain = int(fps * MIN_SUSTAIN_SEC)

THRESHOLD = 7  # how big should the trough be?


# Detecting peaks and troughs for foot-strike:
PROMINENCE = 10
DISTANCE = 15
        

# Stride length: 
STEP_RATIO = 1



# Parameters for if arm is outstretched during the run-up: 
ONSET_OFFSET_MS = 150
OFFSET_BEFORE_RELEASE_MS = 100
WINDOW_BEFORE_RELEASE_MS = 2500
REQUIRED_ABOVE_ANGLE_MS = 150
THRESHOLD_ANGLE = 150
MIN_MAX_DURATION_FRAMES = 2



# Functions: 
def smooth_series(series, window_size=5):
    return pd.Series(series).rolling(window=window_size, min_periods=1, center=True).mean().tolist()



# low butterworth filter
def butter_lowpass_filter(signal, cutoff=5, order=1, fs=30):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, signal)

    

def calculate_angle(ax, ay, bx, by, cx, cy):
    if any(pd.isna([ax, ay, bx, by, cx, cy])):
        return np.nan
    v1, v2 = np.array([ax - bx, ay - by]), np.array([cx - bx, cy - by])
    norm = np.linalg.norm(v1) * np.linalg.norm(v2)
    if norm == 0:
        return np.nan
    angle = np.arccos(np.clip(np.dot(v1, v2) / norm, -1.0, 1.0))
    return np.degrees(angle)

def frame_diff(x, y):
    return [0] + [np.linalg.norm([x[i] - x[i - 1], y[i] - y[i - 1]]) for i in range(1, len(x))]



# Distance function
def dist(wrist_x, wrist_y, obj_x, obj_y):
    if any(pd.isna([wrist_x, wrist_y, obj_x, obj_y])):
        return np.nan
    return np.linalg.norm([wrist_x - obj_x, wrist_y - obj_y])
    



# Split session_name into components
def parse_session_name(session_name):
    parts = session_name.split("_")
    if len(parts) == 5:
        school, class_id, condition, subj_id, throw_nr = parts
    elif len(parts) == 4:
        school = None
        class_id, condition, subj_id, throw_nr = parts
    else:
        return None, None, None, None, None
    try:
        throw_nr = int(str(throw_nr).lstrip("0"))
    except Exception:
        throw_nr = None
    return school, class_id, condition, subj_id, throw_nr



def find_sustained_increases(signal, deriv, threshold, min_sustain=100, min_delta=100, min_fraction=MIN_FRACTION, min_start=20):
        i = min_start
        N = len(signal)
        while i <= N - min_sustain - 1:
            if deriv[i] > threshold:
                window = signal[i:i+min_sustain+1]
                diffs = np.diff(window)
                fraction_increasing = np.mean(diffs > 0)
                sustained = fraction_increasing >= min_fraction
                total_increase = window[-1] - window[0]
                if sustained and total_increase > min_delta:
                    return np.array([i])
            i += 1
        return np.array([])
    



    

# Loop through files: 
for csv_file in tqdm(csv_files, desc="Processing CSVs"):
    session_name = csv_file.stem.replace("_merged", "")

    school, class_id, condition, subj_id, throw_nr = parse_session_name(session_name)

    if school and school.startswith(("BK", "NE")):
        fps = 25
    else:
        fps = 30

        
    df = pd.read_csv(csv_file)
    person_ids = df["New_ID"].unique()

    
    for person_id in person_ids:
        person_df = df[df["New_ID"] == person_id].copy()
        if person_df.empty:
            continue

        
        
         
        # Prepare plot data
        plot_df = person_df.reset_index(drop=True)
        
        
        # Smoothing the signals: 
        # === Step 1: Setup ===
        joint_names = ["ankle", "knee", "hip", "shoulder", "elbow", "wrist"]
        sides = ["left", "right"]
        axes = ["x", "y"]
        
        # Initialize dictionary to collect all coordinate time series
        coords = {f"{side}_{joint}_{axis}": [] for side in sides for joint in joint_names for axis in axes}
        
        # === Step 2: Fill from plot_df row by row ===
        for row in plot_df.itertuples():
            row_data = row._asdict()
            for key in coords:
                coords[key].append(row_data.get(key, np.nan))
        
        # Define joint motion types
        slow_joints = ["hip", "shoulder"]
        fast_joints = ["ankle", "wrist", "elbow", "Tail", "Handle", "Tip"]
        
        # Apply Butterworth filter with joint-specific cutoffs
        for key in coords:
            signal = coords[key]
            
            if len(signal) > 5 and not all(pd.isna(signal)):
                joint_name = key.split("_")[1] if "_" in key else key  # Extract joint name
        
                # Select cutoff frequency based on joint type
                if any(j in joint_name for j in slow_joints):
                    cutoff = 3
                elif any(j in joint_name for j in fast_joints):
                    cutoff = 7
                else:
                    cutoff = 5
        
                # Try filtering
                try:
                    coords[key] = butter_lowpass_filter(np.array(signal), cutoff=cutoff, order=1, fs=30)
                except Exception as e:
                    print(f"⚠️ Skipping filter for {key} due to error: {e}")
        
        # === Step 4: Convert to DataFrame ===
        coords_df = pd.DataFrame(coords)
        
        # Optionally add frame column if needed
        coords_df["frame"] = plot_df["Frame"].values
        frame_to_row = {frame: i for i, frame in enumerate(coords_df["frame"])}
        
        
        # Add center coordinates of Tail, Handle, Tip - x and y: 
        # List of javelin parts
        parts = ["Tip", "Handle", "Tail"]
        missing_parts = []
        
        # Add columns, fill with NaN if missing
        for part in parts:
            x_col = f"{part}_center_x"
            y_col = f"{part}_center_y"
            if x_col not in plot_df.columns or y_col not in plot_df.columns:
                missing_parts.append(part)
            coords_df[x_col] = plot_df.get(x_col, pd.Series([np.nan]*len(plot_df))).values
            coords_df[y_col] = plot_df.get(y_col, pd.Series([np.nan]*len(plot_df))).values
        
        if len(missing_parts) == len(parts):
            print(f"⚠️ Warning: ALL javelin parts (Tip, Handle, Tail) missing in {session_name} (will fill with NaN).")
        
        # --- NEW: only compute center if at least TWO points are present ---
        def mean_if_at_least_two(row, cols):
            values = row[cols].values
            not_nan = np.isfinite(values)
            if np.sum(not_nan) >= 2:
                return np.nanmean(values)
            else:
                return np.nan
        
        x_cols = [f"{part}_center_x" for part in parts]
        y_cols = [f"{part}_center_y" for part in parts]
        
        coords_df["javelin_center_x"] = coords_df.apply(lambda row: mean_if_at_least_two(row, x_cols), axis=1)
        coords_df["javelin_center_y"] = coords_df.apply(lambda row: mean_if_at_least_two(row, y_cols), axis=1)
        
        # Distance from javelin center to wrists (same as before)
        coords_df["dist_javelin_to_right_wrist"] = np.sqrt(
            (coords_df["javelin_center_x"] - coords_df["right_wrist_x"])**2 +
            (coords_df["javelin_center_y"] - coords_df["right_wrist_y"])**2
        )
        coords_df["dist_javelin_to_left_wrist"] = np.sqrt(
            (coords_df["javelin_center_x"] - coords_df["left_wrist_x"])**2 +
            (coords_df["javelin_center_y"] - coords_df["left_wrist_y"])**2
        )
        
        
        
        # Find instant when javelin leasves the hand:     
        # First derivate to get the sharp increase in javelin to wrist:
        left_signal  = coords_df["dist_javelin_to_left_wrist"].values
        right_signal = coords_df["dist_javelin_to_right_wrist"].values
        
        left_deriv  = np.diff(left_signal, prepend=left_signal[0])
        right_deriv = np.diff(right_signal, prepend=right_signal[0])
        
        
        left_sustained_indices  = find_sustained_increases(
            left_signal, left_deriv, threshold=THRESHOLD, min_sustain=min_sustain, min_delta=MIN_DELTA)
        right_sustained_indices = find_sustained_increases(
            right_signal, right_deriv, threshold=THRESHOLD, min_sustain=min_sustain, min_delta=MIN_DELTA)
        
        
        release_indices = []
        if len(left_sustained_indices) > 0 and len(right_sustained_indices) > 0:
            release_idx = int(np.mean([left_sustained_indices[0], right_sustained_indices[0]]))
        elif len(left_sustained_indices) > 0:
            release_idx = left_sustained_indices[0]
        elif len(right_sustained_indices) > 0:
            release_idx = right_sustained_indices[0]
        else:
            release_idx = None
        
        if release_idx is None:
            print(f"No release event detected for {session_name} / person {person_id}. Skipping.")
            continue
                
            
        
        # Calculate slope for both hips
        x = np.arange(len(coords_df))  # Frame indices
        
        # Mean hip x for each frame (averaging left and right hip)
        hip_x_mean = (coords_df["left_hip_x"] + coords_df["right_hip_x"]) / 2
        
        # Fit a line: slope tells direction
        slope, intercept, r_value, p_value, std_err = linregress(x, hip_x_mean)
        
        if slope > 0:
            direction = "to_Right"
        elif slope < 0:
            direction = "to_Left"
        else:
            direction = "No Movement"
        
        
        
        # Calculate hip to ankle distance: 
        coords_df["left_ankle_rel_x"] = coords_df["left_hip_x"] - coords_df["left_ankle_x"]
        coords_df["right_ankle_rel_x"] = coords_df["right_hip_x"] - coords_df["right_ankle_x"]
                    
        left_diff = coords_df["left_ankle_rel_x"].values
        right_diff = coords_df["right_ankle_rel_x"].values

        PROMINENCE = 10
        DISTANCE = 15

        
        # Detect extrema type depending on slope
        if slope > 0:
            # Detect minima (lows)
            left_extrema, _ = find_peaks(-left_diff, distance=DISTANCE, prominence=PROMINENCE)
            right_extrema, _ = find_peaks(-right_diff, distance=DISTANCE, prominence=PROMINENCE)
            extrema_label = "Minima (before release)"
        elif slope < 0:
            # Detect maxima (highs)
            left_extrema, _ = find_peaks(left_diff, distance=DISTANCE, prominence=PROMINENCE)
            right_extrema, _ = find_peaks(right_diff, distance=DISTANCE, prominence=PROMINENCE)
            extrema_label = "Maxima (before release)"
        else:
            left_extrema, right_extrema = np.array([]), np.array([])
            extrema_label = "No movement"
        
        # Only those BEFORE the release event
        if release_idx is not None:
            left_extrema_before = left_extrema[left_extrema < release_idx]
            right_extrema_before = right_extrema[right_extrema < release_idx]
        else:
            left_extrema_before, right_extrema_before = [], []
        
        
        
        # Calculations of joint angles:
        
        coords_df["elbow_l"] = coords_df.apply(
            lambda row: calculate_angle(row["left_wrist_x"], row["left_wrist_y"],
                                         row["left_elbow_x"], row["left_elbow_y"],
                                         row["left_shoulder_x"], row["left_shoulder_y"]), axis=1)
        
        coords_df["elbow_r"] = coords_df.apply(
            lambda row: calculate_angle(row["right_wrist_x"], row["right_wrist_y"],
                                         row["right_elbow_x"], row["right_elbow_y"],
                                         row["right_shoulder_x"], row["right_shoulder_y"]), axis=1)
        
        coords_df["knee_l"] = coords_df.apply(
            lambda row: calculate_angle(row["left_ankle_x"], row["left_ankle_y"],
                                         row["left_knee_x"], row["left_knee_y"],
                                         row["left_hip_x"], row["left_hip_y"]), axis=1)
        
        coords_df["knee_r"] = coords_df.apply(
            lambda row: calculate_angle(row["right_ankle_x"], row["right_ankle_y"],
                                         row["right_knee_x"], row["right_knee_y"],
                                         row["right_hip_x"], row["right_hip_y"]), axis=1)
        
        
        
        
        # Calculate stride length: 
        # leg length: 
        leg_lengths_px = []
        for i, row in coords_df.iterrows():
            dists = []
            if row['knee_r'] > 170:
                dx_r = abs(row['right_hip_x'] - row['right_ankle_x'])
                dy_r = abs(row['right_hip_y'] - row['right_ankle_y'])
                d_r = np.sqrt(dx_r**2 + dy_r**2)
                dists.append(d_r)
            if row['knee_l'] > 170:
                dx_l = abs(row['left_hip_x'] - row['left_ankle_x'])
                dy_l = abs(row['left_hip_y'] - row['left_ankle_y'])
                d_l = np.sqrt(dx_l**2 + dy_l**2)
                dists.append(d_l)
            if dists:
                leg_lengths_px.append(np.mean(dists))
        leg_length_px = np.nanmedian(leg_lengths_px)
        
        
        
        
        # 1. Gather events (frame idx, x, y, side)
        contacts = []
        for idx in left_extrema_before:
            contacts.append((idx, coords_df.loc[idx, 'left_ankle_x'], coords_df.loc[idx, 'left_ankle_y'], 'L'))
        for idx in right_extrema_before:
            contacts.append((idx, coords_df.loc[idx, 'right_ankle_x'], coords_df.loc[idx, 'right_ankle_y'], 'R'))
        
        # 2. Sort by frame index (time)
        contacts.sort(key=lambda x: x[0])  # ascending by frame/time
        
        # 3. Only keep those before release, as above (already done if you used *_before arrays)
        # Optional: Go backwards from last before release
        contacts = [c for c in contacts if c[0] < release_idx]
        
        # 4. Calculate strides only for alternating sides (e.g. L→R, R→L)
        step_lengths_px = []
        step_types = []
        stride_pairs = []  # For optional annotation
        
        for i in range(len(contacts)-1, 0, -1):  # backwards: closest before release to earliest
            idx1, x1, y1, side1 = contacts[i]
            idx0, x0, y0, side0 = contacts[i-1]
            if side1 != side0:
                # Only if side alternates!
                step_length = np.sqrt((x1-x0)**2 + (y1-y0)**2)
                step_lengths_px.append(step_length)
                ratio = step_length / leg_length_px
                if ratio < STEP_RATIO:
                    step_types.append("short")
                else:
                    step_types.append("long")
                stride_pairs.append((idx0, idx1, side0, side1))
        
        
        # Reverse to be time-ordered (earliest to latest)
        step_lengths_px = step_lengths_px[::-1]
        step_types      = step_types[::-1]
        stride_pairs    = stride_pairs[::-1]
        
        # === DEBUG OUTPUT: Stride Events Table ===
        stride_table = []
        for i, ((idx0, idx1, side0, side1), step_type, step_length) in enumerate(zip(stride_pairs, step_types, step_lengths_px)):
            stride_table.append({
                "i": i,
                "from_frame": int(idx0),
                "to_frame": int(idx1),
                "from_side": side0,
                "to_side": side1,
                "step_length_px": float(step_length),
                "leg_length_px": float(leg_length_px),
                "step_length/leg_length": float(step_length / leg_length_px) if leg_length_px else np.nan,
                "type": step_type
            })
        
    
        
        
        
        
        
        # Calculation for assessing if elbow was pointing in the throwing direction
        # PRE window: exactly 500 ms
        
        n_pre_frames = int(np.round(0.500 * fps))
        idx_pre_start = max(0, release_idx - n_pre_frames + 1)
        idx_pre_end = release_idx + 1
        pre_window = coords_df.iloc[idx_pre_start:idx_pre_end]
        
        # POST window: exactly 500 ms
        n_post_frames = int(np.round(0.500 * fps))
        idx_post_start = release_idx
        idx_post_end = min(len(coords_df), release_idx + n_post_frames)
        post_window = coords_df.iloc[idx_post_start:idx_post_end]
        
        if direction == "to_Right":
            throwing_side = "right"
        else:
            throwing_side = "left"
        
        elbow_x_col = f"{throwing_side}_elbow_x"
        wrist_x_col = f"{throwing_side}_wrist_x"
        elbow_angle_col = f"elbow_{throwing_side[0]}"  # 'elbow_r' or 'elbow_l'
        
        
        # 1. Elbow ahead of wrist (20%+)
        if direction == "to_Right":
            ahead_mask = pre_window[elbow_x_col] > pre_window[wrist_x_col]
        else:
            ahead_mask = pre_window[elbow_x_col] < pre_window[wrist_x_col]
        
        frac_ahead = np.mean(ahead_mask)
        criterion_1 = frac_ahead >= 0.20
        
        # 2. Min elbow angle <90° in at least 3% of frames
        frac_min_angle = np.mean(pre_window[elbow_angle_col] < 90)
        criterion_2 = frac_min_angle >= 0.03
        # For min elbow angle < 90° (pre-release)
        num_frames_min_angle = np.sum(pre_window[elbow_angle_col] < 90)
        duration_min_angle_ms = num_frames_min_angle * (1000 / fps)
        
        # 3. Max elbow angle >150° in at least 3% of frames after release
        frac_max_angle = np.mean(post_window[elbow_angle_col] > 150)
        criterion_3 = frac_max_angle >= 0.03
        # For max elbow angle > 150° (post-release)
        num_frames_max_angle = np.sum(post_window[elbow_angle_col] > 150)
        duration_max_angle_ms = num_frames_max_angle * (1000 / fps)
        
        # 4. Positive slope of elbow angle before release 
        slope_win = int(0.300 * fps)
        idx_slope_start = max(0, release_idx - slope_win)
        idx_slope_end = release_idx + 1
        
        elbow_angles_slope = coords_df[elbow_angle_col].iloc[idx_slope_start:idx_slope_end].values
        x_vals = np.arange(len(elbow_angles_slope))
        
        if len(elbow_angles_slope) > 1:
            slope_val, _, _, _, _ = linregress(x_vals, elbow_angles_slope)
            criterion_4 = slope_val > 0
        else:
            slope_val = np.nan
            criterion_4 = False
        
        # For min in pre-window
        tol = 5  # or whatever you use for float tolerance
        min_val = np.nanmin(pre_window[elbow_angle_col])
        min_mask = np.abs(pre_window[elbow_angle_col] - min_val) < tol
        labeled, n_features = label(min_mask)
        durations = [np.sum(labeled == i) for i in range(1, n_features + 1)]
        if any(d >= 3 for d in durations):
            min_angle_valid = min_val
        else:
            min_angle_valid = np.nan
        
        # For max in post-window
        max_val = np.nanmax(post_window[elbow_angle_col])
        max_mask = np.abs(post_window[elbow_angle_col] - max_val) < tol
        labeled, n_features = label(max_mask)
        durations = [np.sum(labeled == i) for i in range(1, n_features + 1)]
        if any(d >= 3 for d in durations):
            max_angle_valid = max_val
        else:
            max_angle_valid = np.nan
        
        
        num_frames_ahead = np.sum(ahead_mask)
        
    
        # Elbow outstretched AND behind shoulder
        
        # Determine correct elbow and shoulder columns based on throwing side
        if direction == "to_Right":
            throwing_side = "right"
            elbow_x_col = "right_elbow_x"
            shoulder_x_col = "right_shoulder_x"
            elbow_angle_col = "elbow_r"
        elif direction == "to_Left":
            throwing_side = "left"
            elbow_x_col = "left_elbow_x"
            shoulder_x_col = "left_shoulder_x"
            elbow_angle_col = "elbow_l"
        else:
            raise ValueError("Unknown throwing direction!")
        
        
  
        
        # Calculate frame indices
        onset_offset_frames = int(np.ceil(ONSET_OFFSET_MS / 1000 * fps))
        offset_frames = int(np.round(OFFSET_BEFORE_RELEASE_MS / 1000 * fps))
        idx_win_end = max(onset_offset_frames, release_idx - offset_frames + 1)
        idx_win_start = max(onset_offset_frames, idx_win_end - int(WINDOW_BEFORE_RELEASE_MS / 1000 * fps))
        
        elbow_series = coords_df[elbow_angle_col].iloc[idx_win_start:idx_win_end].values
        elbow_x_series = coords_df[elbow_x_col].iloc[idx_win_start:idx_win_end].values
        shoulder_x_series = coords_df[shoulder_x_col].iloc[idx_win_start:idx_win_end].values
        
        # 1. Frames with sustained extension AND elbow behind shoulder
        above = elbow_series > THRESHOLD_ANGLE
        if direction == "to_Right":
            behind = elbow_x_series < shoulder_x_series
        elif direction == "to_Left":
            behind = elbow_x_series > shoulder_x_series
        else:
            behind = np.ones_like(elbow_x_series, dtype=bool)  # fallback: don't restrict
        
        above_and_behind = above & behind
        num_above_and_behind = np.sum(above_and_behind)
        duration_above_and_behind_ms = num_above_and_behind * 1000 / fps
        sustained_extension_and_behind = duration_above_and_behind_ms >= REQUIRED_ABOVE_ANGLE_MS

        if sustained_extension_and_behind:
            above_indices = np.where(above_and_behind)[0]
            sustained_start_idx = idx_win_start + above_indices[0]
            sustained_end_idx = idx_win_start + above_indices[-1]
        else:
            sustained_start_idx = sustained_end_idx = np.nan
        
        # 2. Maximum angle reached (descriptive, not a pass/fail)
        max_angle = np.nanmax(elbow_series)
        
        # 3. (Optional/Descriptive) Did max angle persist for at least N frames?
        from scipy.ndimage import label
        tolerance = 5
        max_mask = np.abs(elbow_series - max_angle) < tolerance
        labeled, n_features = label(max_mask)
        max_durations = [np.sum(labeled == i) for i in range(1, n_features + 1)]
        max_angle_streak = max(max_durations) if max_durations else 0
        max_angle_streak_ok = max_angle_streak >= MIN_MAX_DURATION_FRAMES


        if release_idx is None:
            print(f"No release event detected for {session_name} / person {person_id}. Skipping.")
            continue
        summary_row = {
            "session": session_name,
            "person_id": person_id,
            "school": school, 
            "class_id": class_id,
            "condition": condition,
            "subject_id": subj_id,
            "throw_number": throw_nr,
            "release_idx": int(release_idx),
            "direction": direction,
            "fps": fps, 
            "slope": float(slope),
            "leg_length_px": float(leg_length_px),
            "num_strides": len(stride_pairs),
            "stride_from_frames": [int(x[0]) for x in stride_pairs],
            "stride_to_frames": [int(x[1]) for x in stride_pairs],
            "stride_from_sides": [x[2] for x in stride_pairs],
            "stride_to_sides": [x[3] for x in stride_pairs],
            "stride_lengths_px": [float(x) for x in step_lengths_px],
            "stride_types": step_types,
            # Real event-based kinematic fields (replace with your calculated variables)
            "throwing_side": throwing_side,
            "elbow_ahead_of_wrist_ms_pre": float(num_frames_ahead * 1000 / fps),
            "criterion_1_elbow_ahead": bool(criterion_1),
            "elbow_below_90_ms_pre": float(duration_min_angle_ms),
            "criterion_2_min_elbow": bool(criterion_2),
            "elbow_above_150_ms_post": float(duration_max_angle_ms),
            "criterion_3_max_elbow": bool(criterion_3),
            "elbow_angle_slope": float(slope_val) if not np.isnan(slope_val) else np.nan,
            "criterion_4_positive_slope": bool(criterion_4),
            "min_elbow_angle_pre": float(min_angle_valid),
            "max_elbow_angle_post": float(max_angle_valid),
            "max_elbow_angle_pre_release": float(max_angle),
            "max_angle_streak_frames": int(max_angle_streak),
            "sustained_extension": bool(sustained_extension_and_behind),
            "sustained_extension_start_idx": sustained_start_idx,
            "sustained_extension_end_idx": sustained_end_idx,
        }
        all_results.append(summary_row)
    

summary_df = pd.DataFrame(all_results)
summary_df.to_csv(OUTPUT_FILE, index=False)
print(f"Saved summary to {OUTPUT_FILE}")


Processing CSVs:   0%|          | 0/1884 [00:00<?, ?it/s]

No release event detected for NE_8c_ps1_104_02 / person 1. Skipping.
No release event detected for BK_8a_I_23_03 / person 2. Skipping.
No release event detected for BK_8a_I_23_03 / person 3. Skipping.
No release event detected for NE_8c_post_87_03 / person 1. Skipping.
No release event detected for NE_8b_pre_54_04 / person 1. Skipping.
No release event detected for NE_8c_I_86_01 / person 1. Skipping.
⚠️ Skipping filter for left_ankle_x due to error: The length of the input vector x must be greater than padlen, which is 6.
⚠️ Skipping filter for left_ankle_y due to error: The length of the input vector x must be greater than padlen, which is 6.
⚠️ Skipping filter for left_knee_x due to error: The length of the input vector x must be greater than padlen, which is 6.
⚠️ Skipping filter for left_knee_y due to error: The length of the input vector x must be greater than padlen, which is 6.
⚠️ Skipping filter for left_hip_x due to error: The length of the input vector x must be greater than 

/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


No release event detected for BK_8c_ps1_67_01 / person 1. Skipping.
No release event detected for NE_8c_ps1_100_02 / person 1. Skipping.
No release event detected for NE_8a_I_16_02 / person 1. Skipping.
No release event detected for NE_8c_I_82_01 / person 1. Skipping.
No release event detected for NE_8a_I_03_04 / person 1. Skipping.
No release event detected for NE_8c_pre_90_04 / person 1. Skipping.
No release event detected for BK_8a_I_01_02 / person 2. Skipping.
No release event detected for BK_8a_pre_03_04 / person 1. Skipping.
No release event detected for BK_8c_ps1_73_04 / person 1. Skipping.
No release event detected for BK_8c_ps4_69_01 / person 2. Skipping.
No release event detected for BK_8b_ps2_32_01 / person 1. Skipping.
No release event detected for NE_8b_pre_52_02 / person 1. Skipping.
No release event detected for NE_8c_pre_87_04 / person 1. Skipping.


/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


...

/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


No release event detected for BK_8c_post_66_04 / person 2. Skipping.
No release event detected for BK_8c_post_66_04 / person 3. Skipping.
No release event detected for BK_8c_I_72_02 / person 1. Skipping.
No release event detected for BK_8a_post_06_04 / person 1. Skipping.
No release event detected for NE_8a_ps1_06_04 / person 1. Skipping.
No release event detected for NE_8c_ps2_103_03 / person 1. Skipping.
No release event detected for BK_8b_ps1_48_03 / person 1. Skipping.
No release event detected for BK_8b_ps1_48_03 / person 2. Skipping.
⚠️ Skipping filter for left_ankle_x due to error: The length of the input vector x must be greater than padlen, which is 6.
⚠️ Skipping filter for left_ankle_y due to error: The length of the input vector x must be greater than padlen, which is 6.
⚠️ Skipping filter for left_knee_x due to error: The length of the input vector x must be greater than padlen, which is 6.
⚠️ Skipping filter for left_knee_y due to error: The length of the input vector x m

/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


No release event detected for BK_8b_pre_34_03 / person 1. Skipping.
No release event detected for NE_8a_ps3_09_04 / person 1. Skipping.
No release event detected for BK_8c_pre_67_03 / person 2. Skipping.
No release event detected for BK_8a_ps4_04_04 / person 2. Skipping.
No release event detected for BK_8a_ps4_04_04 / person 3. Skipping.
No release event detected for NE_8b_post_48_02 / person 1. Skipping.
No release event detected for BK_8c_I_73_01 / person 1. Skipping.
No release event detected for NE_8a_ps2_15_02 / person 1. Skipping.
No release event detected for NE_8c_ps3_82_01 / person 1. Skipping.
No release event detected for BK_8c_post_71_04 / person 2. Skipping.
No release event detected for BK_8c_post_71_04 / person 4. Skipping.
No release event detected for BK_8b_I_55_03 / person 2. Skipping.
No release event detected for BK_8b_I_55_03 / person 3. Skipping.
No release event detected for BK_8b_I_55_03 / person 4. Skipping.
No release event detected for BK_8a_ps3_10_02 / perso

/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
